In [ ]:
pip install langchain_community faiss-cpu openai langchain_community langchain-text-splitters langchain-openai

In [ ]:
pip install chromadb

In [ ]:
pip install langchain_chroma

In [ ]:
import os

OPENAI_API_KEY= "sk------"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
#@title Create sample vector db- Chroma

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings


In [ ]:
#@title load the data

loader = TextLoader('/content/speech.txt')
docs = loader.load()

In [ ]:
#@title split the data

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

In [ ]:
#@title convert to vector store

embeddings = OpenAIEmbeddings(model ="text-embedding-3-small", openai_api_key= OPENAI_API_KEY)

vector_db = Chroma.from_documents(documents= splits, embedding= embeddings)
vector_db

In [ ]:
#@title query the vector db

query ="what is the main idea behind Continuous Skip-gram Model"
res = vector_db.similarity_search(query)
res

[Document(id='9427ffda-7433-4ebf-aa97-3ee3b308e8ae', metadata={'source': '/content/speech.txt'}, page_content="b) Continuous Skip-gram Model\nIdea: Predicts the surrounding context words given the current word. It's essentially the inverse of CBOW."),
 Document(id='32c8ee67-f12e-458e-91de-e380f5a6b9a2', metadata={'source': '/content/speech.txt'}, page_content="b) Continuous Skip-gram Model\nIdea: Predicts the surrounding context words given the current word. It's essentially the inverse of CBOW."),
 Document(id='a24496ce-81d4-444b-9e96-702e3f72f8ca', metadata={'source': '/content/speech.txt'}, page_content='Performance: The paper found that Skip-gram generally works better for capturing semantic relationships, especially with large datasets, while CBOW can be faster to train.'),
 Document(id='bc24a62c-6f86-4101-b6c9-3ebdd5f86e17', metadata={'source': '/content/speech.txt'}, page_content='Performance: The paper found that Skip-gram generally works better for capturing semantic relations

In [ ]:
res[0].page_content

"b) Continuous Skip-gram Model\nIdea: Predicts the surrounding context words given the current word. It's essentially the inverse of CBOW."

In [ ]:
#@title Save vector store db in local

vector_db = Chroma.from_documents(documents= splits, embedding= embeddings, persist_directory="/content/chroma_db")

In [ ]:
#@title load the vector db

new_db = Chroma(persist_directory="/content/chroma_db", embedding_function= embeddings)

In [ ]:
#@title user the loaded vector db

result = new_db.similarity_search(query)
result[3].page_content

'simplified model architectures and clever optimization techniques like hierarchical softmax and negative sampling, which enabled training on unprecedentedly large text corpora.'

In [ ]:
 #@title similarity search with score

result2 = new_db.similarity_search_with_score(query)
result2[0]

(Document(id='00b16cd0-7ed1-4f78-babf-1cde2c045842', metadata={'source': '/content/speech.txt'}, page_content="b) Continuous Skip-gram Model\nIdea: Predicts the surrounding context words given the current word. It's essentially the inverse of CBOW."),
 0.4862409234046936)

In [ ]:
#@title Retriever

retriever = vector_db.as_retriever()
retriever.invoke(query)[0].page_content

"b) Continuous Skip-gram Model\nIdea: Predicts the surrounding context words given the current word. It's essentially the inverse of CBOW."

In [ ]:
# https://python.langchain.com/docs/integrations/vectorstores/